In [ ]:
import ast
import pandas as pd

In [ ]:
df_unichain = pd.read_csv("/home/tobias/personal-dex-trading/out/data/20250801_142508_unichain_uniswap_v4_blocks.csv", index_col="timestamp")
df_unichain.index = pd.to_datetime(df_unichain.index, unit="s")  # seconds -> datetime
df_unichain.index += pd.Timedelta(hours=2)
df_unichain

In [ ]:
BINANCE_FEE = 0.001 # 0.1% for Regular User https://www.binance.com/en/fee/trading

def _top_price(side_str: str) -> float | None:
    """Return the first price in a stringified list of bids/asks."""
    try:
        side = ast.literal_eval(side_str)
        return float(side[0][0]) if side else None
    except Exception:
        return None

def load_binance(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)           # columns: time, bids, asks
    df["best_bid"] = df["bids"].apply(_top_price)
    df["best_ask"] = df["asks"].apply(_top_price)

    return df[["time", "best_bid", "best_ask"]].sort_values("time")

def apply_fee(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds columns 'best_bid_with_fee' and 'best_ask_with_fee' to account for the Binance trading fee.
    """
    df["best_bid_with_fee"] = df["best_bid"] * (1 - BINANCE_FEE)  # Subtract fee for bids
    df["best_ask_with_fee"] = df["best_ask"] * (1 + BINANCE_FEE)  # Add fee for asks

    return df


binance_df = load_binance("/home/tobias/personal-dex-trading/out/data/20250801_142514_binance_ws_orderbook.csv")
binance_df["time"] = pd.to_datetime(binance_df["time"])
binance_df = apply_fee(binance_df)
binance_df

In [ ]:
merged_df =  pd.merge_asof(
    binance_df.sort_values("time"),
    df_unichain.sort_index(),
    left_on="time",
    right_index=True,
    direction="backward"
)
merged_df["bid_0.00001_weth"] = (merged_df["token1_outputs"].apply(ast.literal_eval).apply(lambda x: x[0][0] if isinstance(x, list) else None) / 10**6)
merged_df["ask_0.00001_weth"] = (merged_df["token1_inputs"].apply(ast.literal_eval).apply(lambda x: x[0][0] if isinstance(x, list) else None) / 10**6)
merged_df

In [ ]:
import plotly.express as px

long_df = merged_df.melt(id_vars="time",
                         value_vars=["best_bid", "best_ask", "bid_0.00001_weth", "ask_0.00001_weth"],
                         var_name="Type",
                         value_name="Price")

# Plot with Plotly Express
fig = px.line(
    long_df,
    x="time",
    y="Price",
    color="Type",
    markers=True,
    title="Best Bids/Asks vs Bid/Ask for 0.00001 WETH",
    labels={"time": "Time", "Price": "Price", "Type": "Legend"}
)

# Update layout for better readability
fig.update_layout(
    xaxis=dict(showgrid=True, tickformat='%H:%M:%S'),
    yaxis=dict(showgrid=True),
    hovermode="x unified"  # Show hover labels for all lines
)

# Show the plot
fig.show()

In [ ]:
BINANCE_CSV   = "/home/tobias/personal-dex-trading/out/data/order_book.csv"
UNICHAIN_CSV  = "/home/tobias/personal-dex-trading/out/data/block_data.csv"

def _top_price(side_str: str) -> float | None:
    """Return the first price in a stringified list of bids/asks."""
    try:
        side = ast.literal_eval(side_str)
        return float(side[0][0]) if side else None
    except Exception:
        return None

def load_binance(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)           # columns: time, bids, asks
    df["best_bid"] = df["bids"].apply(_top_price)
    df["best_ask"] = df["asks"].apply(_top_price)

    return df[["time", "best_bid", "best_ask"]].sort_values("time")

def load_unichain(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, index_col="timestamp")
    df.index = pd.to_datetime(df.index, unit="s")  # seconds -> datetime
    df.index += pd.Timedelta(hours=2)              # match binance tz

    # Pivot so each input size has its own column of output_value
    df_pivot = (
        df.reset_index()
          .pivot_table(index="timestamp",
                       columns="input_value",
                       values="output_value")
          .rename_axis(None, axis=1)
          .sort_index()
    )

    # Convert WETH→USDC **prices** (USDC per 1 WETH) for each size
    price_cols = {}
    for in_size in df_pivot.columns:
        price_cols[f"uni_px_{in_size}"] = df_pivot[in_size] / in_size

    df_px = pd.concat(price_cols, axis=1)
    return pd.concat([df_pivot, df_px], axis=1).reset_index(names="time")

def merge_dfs(df_bin: pd.DataFrame, df_uni: pd.DataFrame) -> pd.DataFrame:
    df_bin['time'] = pd.to_datetime(df_bin['time'])
    df_uni['time'] = pd.to_datetime(df_uni['time'])
    df_uni_unique = df_uni.drop_duplicates(subset=['time'], keep='first')
    merged_df = pd.merge(
        df_bin,
        df_uni_unique,
        on='time',
        how='left'
    )
    return merged_df

def add_arbitrage_cols(df: pd.DataFrame) -> pd.DataFrame:
    """
    A. Long-Binance / Short-Unichain:
         buy WETH on Binance @ best_ask, immediately sell on Unichain.
    B. Long-Unichain / Short-Binance:
         buy WETH on Unichain (reverse swap) and sell on Binance @ best_bid.
       For (B) we only have the forward quote; a rough proxy is to use
       best_bid / uni_px   (not perfect – use reverse quote if available).
    """
    # We’ll take the smallest quoted size (0.0001 WETH) as representative:
    uni_price = df["uni_px_0.0001"]

    df["edge_binance→uni"] = uni_price - df["best_ask"]   # positive ⇒ profit
    df["edge_uni→binance"] = df["best_bid"] - uni_price   # positive ⇒ profit
    return df

def main():
    df_bin = load_binance(BINANCE_CSV)
    df_uni = load_unichain(UNICHAIN_CSV)

    merged = merge_dfs(df_bin, df_uni)
    merged = add_arbitrage_cols(merged)

    # Save or plot if you wish:
    #merged.to_csv("merged_for_analysis.csv", index=False)
    return merged

In [ ]:
merged = main()
merged